# Gene_cls_1

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

import time
t0start = time.time()
import os
# Main results will be stored here: 
df_scores = pd.DataFrame()
import seaborn as sns

sns.set()
%matplotlib inline

In [3]:
data_path="/workspace/tripx/MCS/xai_causality/dataset/uci_gene_exp/TCGA-PANCAN-HiSeq-801x20531/data.csv"
label_path="/workspace/tripx/MCS/xai_causality/dataset/uci_gene_exp/TCGA-PANCAN-HiSeq-801x20531/labels.csv"

In [4]:
data_df = pd.read_csv(data_path)

In [5]:
data_df.describe()

,gene_0,gene_1,gene_2,gene_3,gene_4,gene_5,gene_6,gene_7,gene_8,gene_9,...,gene_20521,gene_20522,gene_20523,gene_20524,gene_20525,gene_20526,gene_20527,gene_20528,gene_20529,gene_20530
count,801.000000,801.000000,801.000000,801.000000,801.000000,801.0,801.000000,801.000000,801.000000,801.000000,...,801.000000,801.000000,801.000000,801.000000,801.000000,801.000000,801.000000,801.000000,801.000000,801.000000
mean,0.026642,3.010909,3.095350,6.722305,9.813612,0.0,7.405509,0.499882,0.016744,0.013428,...,5.896573,8.765891,10.056252,4.847727,9.741987,11.742228,10.155271,9.590726,5.528177,0.095411
std,0.136850,1.200828,1.065601,0.638819,0.506537,0.0,1.108237,0.508799,0.133635,0.204722,...,0.746399,0.603176,0.379278,2.382728,0.533898,0.670371,0.580569,0.563849,2.073859,0.364529
min,0.000000,0.000000,0.000000,5.009284,8.435999,0.0,3.930747,0.000000,0.000000,0.000000,...,2.853517,6.678368,8.669456,0.000000,7.974942,9.045255,7.530141,7.864533,0.593975,0.000000
25%,0.000000,2.299039,2.390365,6.303346,9.464466,0.0,6.676042,0.000000,0.000000,0.000000,...,5.454926,8.383834,9.826027,3.130750,9.400747,11.315857,9.836525,9.244219,4.092385,0.000000
50%,0.000000,3.143687,3.127006,6.655893,9.791599,0.0,7.450114,0.443076,0.000000,0.000000,...,5.972582,8.784144,10.066385,5.444935,9.784524,11.749802,10.191207,9.566511,5.218618,0.000000
75%,0.000000,3.883484,3.802534,7.038447,10.142324,0.0,8.121984,0.789354,0.000000,0.000000,...,6.411292,9.147136,10.299025,6.637412,10.082269,12.177852,10.578561,9.917888,6.876382,0.000000
max,1.482332,6.237034,6.063484,10.129528,11.355621,0.0,10.718190,2.779008,1.785592,4.067604,...,7.771054,11.105431,11.318243,9.207495,11.811632,13.715361,11.675653,12.813320,11.205836,5.254133


In [6]:
data_df.head()

,Unnamed: 0,gene_0,gene_1,gene_2,gene_3,gene_4,gene_5,gene_6,gene_7,gene_8,...,gene_20521,gene_20522,gene_20523,gene_20524,gene_20525,gene_20526,gene_20527,gene_20528,gene_20529,gene_20530
0,sample_0,0.0,2.017209,3.265527,5.478487,10.431999,0.0,7.175175,0.591871,0.0,...,4.926711,8.210257,9.723516,7.220030,9.119813,12.003135,9.650743,8.921326,5.286759,0.0
1,sample_1,0.0,0.592732,1.588421,7.586157,9.623011,0.0,6.816049,0.000000,0.0,...,4.593372,7.323865,9.740931,6.256586,8.381612,12.674552,10.517059,9.397854,2.094168,0.0
2,sample_2,0.0,3.511759,4.327199,6.881787,9.870730,0.0,6.972130,0.452595,0.0,...,5.125213,8.127123,10.908640,5.401607,9.911597,9.045255,9.788359,10.090470,1.683023,0.0
3,sample_3,0.0,3.663618,4.507649,6.659068,10.196184,0.0,7.843375,0.434882,0.0,...,6.076566,8.792959,10.141520,8.942805,9.601208,11.392682,9.694814,9.684365,3.292001,0.0
4,sample_4,0.0,2.655741,2.821547,6.539454,9.738265,0.0,6.566967,0.360982,0.0,...,5.996032,8.891425,10.373790,7.181162,9.846910,11.922439,9.217749,9.461191,5.110372,0.0


In [7]:
def data_description(df):
    print("Data description")
    print(f"Total number of records {df.shape[0]}")
    print(f'number of features {df.shape[1]}\n\n')
    columns = df.columns
    data_type = []
    
    # Get the datatype of features
    for col in df.columns:
        data_type.append(df[col].dtype)
        if df[col].dtype != float:
            print(col)
        
    n_uni = df.nunique()
    # Number of NaN values
    n_miss = df.isna().sum()
    
    names = list(zip(columns, data_type, n_uni, n_miss))
    variable_desc = pd.DataFrame(names, columns=["Name","Type","Unique levels","Missing"])
    print(variable_desc)

In [8]:
def count_non_zero_values(df):
    df = df.drop(['Unnamed: 0'], axis=1)
    # get data about non-zero values
    min_cells_non_zero = df.gt(0).sum(axis=1).min()
    max_cells_non_zero = df.gt(0).sum(axis=1).max()
    sum_non_zero_values = df.sum().sum()
    count_non_zero_values = df.gt(0).sum().sum()
    average_non_zero_per_gene = df[df.gt(0)].count(axis = 1).mean()

    print(f"Each cell has at least {min_cells_non_zero} genes with non-zero accessibility values and a maximum of {max_cells_non_zero}.")
    print(f"On average there are {round(average_non_zero_per_gene)} genes with non-zero accessibility values in each cell.")
    print(f"The average non-zero value is about {sum_non_zero_values / count_non_zero_values:.2f}.")

    # investigate standard deviation of features
    std_dev_of_genes = df.std()

    # ignore genes that are only accessible in a single cell
    std_dev_of_genes_without_singles = std_dev_of_genes[df.gt(0).sum().gt(1)]
    print(f"The standard deviation of our features is between {std_dev_of_genes_without_singles.min():.2f} and {std_dev_of_genes_without_singles.max():.2f}.\nThe average standard deviation is {std_dev_of_genes_without_singles.mean():.2f}")


In [9]:
data_description(data_df)

Data description
Total number of records 801
number of features 20532


Unnamed: 0
             Name     Type  Unique levels  Missing
0      Unnamed: 0   object            801        0
1          gene_0  float64             36        0
2          gene_1  float64            770        0
3          gene_2  float64            793        0
4          gene_3  float64            800        0
...           ...      ...            ...      ...
20527  gene_20526  float64            800        0
20528  gene_20527  float64            801        0
20529  gene_20528  float64            800        0
20530  gene_20529  float64            800        0
20531  gene_20530  float64             99        0

[20532 rows x 4 columns]


In [10]:
count_non_zero_values(data_df)

Each cell has at least 15939 genes with non-zero accessibility values and a maximum of 18909.
On average there are 17612 genes with non-zero accessibility values in each cell.
The average non-zero value is about 7.51.
The standard deviation of our features is between 0.02 and 6.69.
The average standard deviation is 1.12


In [11]:
data_df.head()

,Unnamed: 0,gene_0,gene_1,gene_2,gene_3,gene_4,gene_5,gene_6,gene_7,gene_8,...,gene_20521,gene_20522,gene_20523,gene_20524,gene_20525,gene_20526,gene_20527,gene_20528,gene_20529,gene_20530
0,sample_0,0.0,2.017209,3.265527,5.478487,10.431999,0.0,7.175175,0.591871,0.0,...,4.926711,8.210257,9.723516,7.220030,9.119813,12.003135,9.650743,8.921326,5.286759,0.0
1,sample_1,0.0,0.592732,1.588421,7.586157,9.623011,0.0,6.816049,0.000000,0.0,...,4.593372,7.323865,9.740931,6.256586,8.381612,12.674552,10.517059,9.397854,2.094168,0.0
2,sample_2,0.0,3.511759,4.327199,6.881787,9.870730,0.0,6.972130,0.452595,0.0,...,5.125213,8.127123,10.908640,5.401607,9.911597,9.045255,9.788359,10.090470,1.683023,0.0
3,sample_3,0.0,3.663618,4.507649,6.659068,10.196184,0.0,7.843375,0.434882,0.0,...,6.076566,8.792959,10.141520,8.942805,9.601208,11.392682,9.694814,9.684365,3.292001,0.0
4,sample_4,0.0,2.655741,2.821547,6.539454,9.738265,0.0,6.566967,0.360982,0.0,...,5.996032,8.891425,10.373790,7.181162,9.846910,11.922439,9.217749,9.461191,5.110372,0.0


In [12]:
# data_df.corr()

In [13]:
label_data_df = pd.read_csv(label_path)

In [14]:
label_data_df = pd.read_csv(label_path)

In [15]:
label_data_df.head()

,Unnamed: 0,Class
0,sample_0,PRAD
1,sample_1,LUAD
2,sample_2,PRAD
3,sample_3,PRAD
4,sample_4,BRCA


In [16]:
labels = label_data_df['Class'].unique()

In [17]:
df_dumpis = pd.get_dummies(label_data_df, columns=['Class'], prefix='', prefix_sep='')
df_labels_new = df_dumpis.groupby('Unnamed: 0').sum()
df_labels_new = df_labels_new.reset_index()

print(df_labels_new)

     Unnamed: 0  BRCA  COAD  KIRC  LUAD  PRAD
0      sample_0     0     0     0     0     1
1      sample_1     0     0     0     1     0
2     sample_10     1     0     0     0     0
3    sample_100     1     0     0     0     0
4    sample_101     0     0     1     0     0
..          ...   ...   ...   ...   ...   ...
796   sample_95     0     0     0     0     1
797   sample_96     0     1     0     0     0
798   sample_97     1     0     0     0     0
799   sample_98     0     0     0     0     1
800   sample_99     1     0     0     0     0

[801 rows x 6 columns]


In [18]:
df_labels_new.head()

,Unnamed: 0,BRCA,COAD,KIRC,LUAD,PRAD
0,sample_0,0,0,0,0,1
1,sample_1,0,0,0,1,0
2,sample_10,1,0,0,0,0
3,sample_100,1,0,0,0,0
4,sample_101,0,0,1,0,0


In [19]:
data_df.head()

,Unnamed: 0,gene_0,gene_1,gene_2,gene_3,gene_4,gene_5,gene_6,gene_7,gene_8,...,gene_20521,gene_20522,gene_20523,gene_20524,gene_20525,gene_20526,gene_20527,gene_20528,gene_20529,gene_20530
0,sample_0,0.0,2.017209,3.265527,5.478487,10.431999,0.0,7.175175,0.591871,0.0,...,4.926711,8.210257,9.723516,7.220030,9.119813,12.003135,9.650743,8.921326,5.286759,0.0
1,sample_1,0.0,0.592732,1.588421,7.586157,9.623011,0.0,6.816049,0.000000,0.0,...,4.593372,7.323865,9.740931,6.256586,8.381612,12.674552,10.517059,9.397854,2.094168,0.0
2,sample_2,0.0,3.511759,4.327199,6.881787,9.870730,0.0,6.972130,0.452595,0.0,...,5.125213,8.127123,10.908640,5.401607,9.911597,9.045255,9.788359,10.090470,1.683023,0.0
3,sample_3,0.0,3.663618,4.507649,6.659068,10.196184,0.0,7.843375,0.434882,0.0,...,6.076566,8.792959,10.141520,8.942805,9.601208,11.392682,9.694814,9.684365,3.292001,0.0
4,sample_4,0.0,2.655741,2.821547,6.539454,9.738265,0.0,6.566967,0.360982,0.0,...,5.996032,8.891425,10.373790,7.181162,9.846910,11.922439,9.217749,9.461191,5.110372,0.0


In [20]:
merge_data = data_df.join(df_labels_new.set_index('Unnamed: 0'), on='Unnamed: 0')

In [ ]:
merge_data = merge_data.drop(['Unnamed: 0'], axis=1)

In [34]:
merge_data

,gene_0,gene_1,gene_2,gene_3,gene_4,gene_5,gene_6,gene_7,gene_8,gene_9,...,gene_20526,gene_20527,gene_20528,gene_20529,gene_20530,BRCA,COAD,KIRC,LUAD,PRAD
0,0.0,2.017209,3.265527,5.478487,10.431999,0.0,7.175175,0.591871,0.0,0.0,...,12.003135,9.650743,8.921326,5.286759,0.000000,0,0,0,0,1
1,0.0,0.592732,1.588421,7.586157,9.623011,0.0,6.816049,0.000000,0.0,0.0,...,12.674552,10.517059,9.397854,2.094168,0.000000,0,0,0,1,0
2,0.0,3.511759,4.327199,6.881787,9.870730,0.0,6.972130,0.452595,0.0,0.0,...,9.045255,9.788359,10.090470,1.683023,0.000000,0,0,0,0,1
3,0.0,3.663618,4.507649,6.659068,10.196184,0.0,7.843375,0.434882,0.0,0.0,...,11.392682,9.694814,9.684365,3.292001,0.000000,0,0,0,0,1
4,0.0,2.655741,2.821547,6.539454,9.738265,0.0,6.566967,0.360982,0.0,0.0,...,11.922439,9.217749,9.461191,5.110372,0.000000,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
796,0.0,1.865642,2.718197,7.350099,10.006003,0.0,6.764792,0.496922,0.0,0.0,...,12.031267,9.813063,10.092770,8.819269,0.000000,1,0,0,0,0
797,0.0,3.942955,4.453807,6.346597,10.056868,0.0,7.320331,0.000000,0.0,0.0,...,11.633422,10.317266,8.745983,9.659081,0.000000,0,0,0,1,0
798,0.0,3.249582,3.707492,8.185901,9.504082,0.0,7.536589,1.811101,0.0,0.0,...,12.180944,10.681194,9.466711,4.677458,0.586693,0,1,0,0,0
799,0.0,2.590339,2.787976,7.318624,9.987136,0.0,9.213464,0.000000,0.0,0.0,...,11.198279,10.335513,10.400581,5.718751,0.000000,0,0,0,0,1


In [27]:
corr_merge_data = merge_data.corr()

In [29]:
corr_merge_data

,gene_0,gene_1,gene_2,gene_3,gene_4,gene_5,gene_6,gene_7,gene_8,gene_9,...,gene_20526,gene_20527,gene_20528,gene_20529,gene_20530,BRCA,COAD,KIRC,LUAD,PRAD
gene_0,1.000000,0.024111,0.027117,-0.080269,-0.028969,NaN,0.051855,-0.053647,-0.024423,-0.012785,...,0.034314,-0.000368,-0.049142,0.041554,-0.027767,-0.086455,-0.010638,0.068706,0.048824,-0.000323
gene_1,0.024111,1.000000,0.533902,0.148348,-0.158555,NaN,0.202855,0.177154,0.023411,0.006425,...,-0.255093,0.303771,0.247959,-0.067474,0.034348,-0.110372,0.116997,-0.241075,0.133781,0.162088
gene_2,0.027117,0.533902,1.000000,0.075988,-0.125376,NaN,0.334900,0.136246,0.013228,-0.045725,...,-0.160446,0.163181,0.066709,0.011765,0.075604,-0.210428,0.044188,-0.286994,0.157269,0.371960
gene_3,-0.080269,0.148348,0.075988,1.000000,0.153666,NaN,-0.007905,-0.084841,-0.033180,0.002434,...,0.092787,-0.000379,0.016733,-0.135829,0.023476,-0.035990,0.570058,-0.282038,-0.070312,-0.042411
gene_4,-0.028969,-0.158555,-0.125376,0.153666,1.000000,NaN,-0.044347,-0.156698,0.082124,0.098267,...,0.011739,-0.434604,-0.108036,-0.007229,0.062752,0.290639,0.077949,-0.342077,-0.029333,-0.054716
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
BRCA,-0.086455,-0.110372,-0.210428,-0.035990,0.290639,NaN,0.068700,0.000032,0.028798,0.068933,...,0.007267,-0.026818,0.252122,0.253156,-0.047453,1.000000,-0.254167,-0.365340,-0.357667,-0.349945
COAD,-0.010638,0.116997,0.044188,0.570058,0.077949,NaN,-0.033546,0.005965,-0.041180,-0.021558,...,0.231957,0.079027,-0.138782,-0.303036,-0.050027,-0.254167,1.000000,-0.155072,-0.151815,-0.148538
KIRC,0.068706,-0.241075,-0.286994,-0.282038,-0.342077,NaN,-0.169441,0.135089,0.028564,-0.030987,...,0.044454,0.039700,-0.133745,-0.097704,0.049584,-0.365340,-0.155072,1.000000,-0.218219,-0.213508
LUAD,0.048824,0.133781,0.157269,-0.070312,-0.029333,NaN,-0.051076,-0.096993,-0.002628,-0.030336,...,0.031836,-0.153953,-0.160702,-0.055942,0.122023,-0.357667,-0.151815,-0.218219,1.000000,-0.209024


In [35]:
merge_data.to_csv("/workspace/tripx/MCS/xai_causality/dataset/uci_gene_exp/merge_data.csv")

# Adult Income


In [1]:
pip install ucimlrepo

Note: you may need to restart the kernel to use updated packages.


In [2]:
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
adult = fetch_ucirepo(id=2) 
  
# data (as pandas dataframes) 
X = adult.data.features 
y = adult.data.targets
print("X: ", X) 
print("Y: ", y)
  
# # metadata 
# print(adult.metadata) 
  
# # variable information 
# print(adult.variables) 


X:         age         workclass  fnlwgt  education  education-num  \
0       39         State-gov   77516  Bachelors             13   
1       50  Self-emp-not-inc   83311  Bachelors             13   
2       38           Private  215646    HS-grad              9   
3       53           Private  234721       11th              7   
4       28           Private  338409  Bachelors             13   
...    ...               ...     ...        ...            ...   
48837   39           Private  215419  Bachelors             13   
48838   64               NaN  321403    HS-grad              9   
48839   38           Private  374983  Bachelors             13   
48840   44           Private   83891  Bachelors             13   
48841   35      Self-emp-inc  182148  Bachelors             13   

           marital-status         occupation    relationship  \
0           Never-married       Adm-clerical   Not-in-family   
1      Married-civ-spouse    Exec-managerial         Husband   
2          

In [3]:
print(adult.variables) 

              name     role         type      demographic  \
0              age  Feature      Integer              Age   
1        workclass  Feature  Categorical           Income   
2           fnlwgt  Feature      Integer             None   
3        education  Feature  Categorical  Education Level   
4    education-num  Feature      Integer  Education Level   
5   marital-status  Feature  Categorical            Other   
6       occupation  Feature  Categorical            Other   
7     relationship  Feature  Categorical            Other   
8             race  Feature  Categorical             Race   
9              sex  Feature       Binary              Sex   
10    capital-gain  Feature      Integer             None   
11    capital-loss  Feature      Integer             None   
12  hours-per-week  Feature      Integer             None   
13  native-country  Feature  Categorical            Other   
14          income   Target       Binary           Income   

                       

In [4]:
# Selection features 
new_data = X[["hours-per-week", "education", "occupation", "workclass", "race", "age", "marital-status", "sex"]]

In [5]:
new_data

,hours-per-week,education,occupation,workclass,race,age,marital-status,sex
0,40,Bachelors,Adm-clerical,State-gov,White,39,Never-married,Male
1,13,Bachelors,Exec-managerial,Self-emp-not-inc,White,50,Married-civ-spouse,Male
2,40,HS-grad,Handlers-cleaners,Private,White,38,Divorced,Male
3,40,11th,Handlers-cleaners,Private,Black,53,Married-civ-spouse,Male
4,40,Bachelors,Prof-specialty,Private,Black,28,Married-civ-spouse,Female
...,...,...,...,...,...,...,...,...
48837,36,Bachelors,Prof-specialty,Private,White,39,Divorced,Female
48838,40,HS-grad,NaN,NaN,Black,64,Widowed,Male
48839,50,Bachelors,Prof-specialty,Private,White,38,Married-civ-spouse,Male
48840,40,Bachelors,Adm-clerical,Private,Asian-Pac-Islander,44,Divorced,Male


In [6]:
y['income']

0         <=50K
1         <=50K
2         <=50K
3         <=50K
4         <=50K
          ...  
48837    <=50K.
48838    <=50K.
48839    <=50K.
48840    <=50K.
48841     >50K.
Name: income, Length: 48842, dtype: object

In [ ]:
# Remove "." in value columns
y['income'] = y['income'].str.strip(".")

In [ ]:
y['income']

In [8]:
new_data["income"] = y["income"].to_list()

/tmp/ipykernel_2925168/1998608528.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_data["income"] = y["income"].to_list()


In [9]:
new_data

,hours-per-week,education,occupation,workclass,race,age,marital-status,sex,income
0,40,Bachelors,Adm-clerical,State-gov,White,39,Never-married,Male,<=50K
1,13,Bachelors,Exec-managerial,Self-emp-not-inc,White,50,Married-civ-spouse,Male,<=50K
2,40,HS-grad,Handlers-cleaners,Private,White,38,Divorced,Male,<=50K
3,40,11th,Handlers-cleaners,Private,Black,53,Married-civ-spouse,Male,<=50K
4,40,Bachelors,Prof-specialty,Private,Black,28,Married-civ-spouse,Female,<=50K
...,...,...,...,...,...,...,...,...,...
48837,36,Bachelors,Prof-specialty,Private,White,39,Divorced,Female,<=50K.
48838,40,HS-grad,NaN,NaN,Black,64,Widowed,Male,<=50K.
48839,50,Bachelors,Prof-specialty,Private,White,38,Married-civ-spouse,Male,<=50K.
48840,40,Bachelors,Adm-clerical,Private,Asian-Pac-Islander,44,Divorced,Male,<=50K.


### Process NaN value 


In [10]:
row_nan_count = new_data.isna().sum(axis=1)
print(row_nan_count)

0        0
1        0
2        0
3        0
4        0
        ..
48837    0
48838    2
48839    0
48840    0
48841    0
Length: 48842, dtype: int64


In [25]:
row_nan_count.value_counts()

0    47876
2      963
1        3
dtype: int64

In [11]:
nonnan_data = new_data.dropna()

In [14]:
nonnan_data

,hours-per-week,education,occupation,workclass,race,age,marital-status,sex,income
0,40,Bachelors,Adm-clerical,State-gov,White,39,Never-married,Male,<=50K
1,13,Bachelors,Exec-managerial,Self-emp-not-inc,White,50,Married-civ-spouse,Male,<=50K
2,40,HS-grad,Handlers-cleaners,Private,White,38,Divorced,Male,<=50K
3,40,11th,Handlers-cleaners,Private,Black,53,Married-civ-spouse,Male,<=50K
4,40,Bachelors,Prof-specialty,Private,Black,28,Married-civ-spouse,Female,<=50K
...,...,...,...,...,...,...,...,...,...
48836,40,Bachelors,Prof-specialty,Private,White,33,Never-married,Male,<=50K.
48837,36,Bachelors,Prof-specialty,Private,White,39,Divorced,Female,<=50K.
48839,50,Bachelors,Prof-specialty,Private,White,38,Married-civ-spouse,Male,<=50K.
48840,40,Bachelors,Adm-clerical,Private,Asian-Pac-Islander,44,Divorced,Male,<=50K.


In [ ]:
import pandas as pd

In [ ]:
nonnan_data

In [30]:
cat_columns = nonnan_data.select_dtypes(['object']).columns 
nonnan_data[cat_columns] = nonnan_data[cat_columns].apply(lambda x: pd.factorize(x)[0])
# convert boolean to numberic values
bool_columns = nonnan_data.select_dtypes(['boolean']).columns
nonnan_data[bool_columns] = nonnan_data[bool_columns].replace({True: 1, False: 0})

/tmp/ipykernel_2636319/2148826127.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nonnan_data[cat_columns] = nonnan_data[cat_columns].apply(lambda x: pd.factorize(x)[0])


In [31]:
nonnan_data

,hours-per-week,education,occupation,workclass,race,age,marital-status,sex,income
0,40,0,0,0,0,39,0,0,0
1,13,0,1,1,0,50,1,0,0
2,40,1,2,2,0,38,2,0,0
3,40,2,2,2,1,53,1,0,0
4,40,0,3,2,1,28,1,1,0
...,...,...,...,...,...,...,...,...,...
48836,40,0,3,2,0,33,0,0,2
48837,36,0,3,2,0,39,2,1,2
48839,50,0,3,2,0,38,1,0,2
48840,40,0,0,2,2,44,2,0,2


In [1]:
nonnan_data.to_csv("/workspace/tripx/MCS/xai_causality/dataset/adult_income_filtered.csv")

NameError: name 'nonnan_data' is not defined

### Breast Cancer

In [1]:
!pip install ucimlrepo

In [1]:
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
breast_cancer = fetch_ucirepo(id=14) 
  
# data (as pandas dataframes) 
X = breast_cancer.data.features 
y = breast_cancer.data.targets 
  
# metadata 
print(breast_cancer.metadata) 
  
# variable information 
print(breast_cancer.variables) 


{'uci_id': 14, 'name': 'Breast Cancer', 'repository_url': 'https://archive.ics.uci.edu/dataset/14/breast+cancer', 'data_url': 'https://archive.ics.uci.edu/static/public/14/data.csv', 'abstract': 'Breast Cancer Data (Restricted Access)', 'area': 'Life Science', 'tasks': ['Classification'], 'characteristics': ['Multivariate'], 'num_instances': 286, 'num_features': 9, 'feature_types': ['Categorical'], 'demographics': ['Age'], 'target_col': ['Class'], 'index_col': None, 'has_missing_values': 'yes', 'missing_values_symbol': 'NaN', 'year_of_dataset_creation': 1988, 'last_updated': 'Mon Aug 28 2023', 'dataset_doi': '10.24432/C51P4M', 'creators': ['Matjaz Zwitter', 'Milan Soklic'], 'intro_paper': None, 'additional_info': {'summary': 'This is one of three domains provided by the Oncology Institute that has repeatedly appeared in the machine learning literature. (See also lymphography and primary-tumor.)\r\n\r\nThis data set includes 201 instances of one class and 85 instances of another class. 

In [3]:
X

,age,menopause,tumor-size,inv-nodes,node-caps,deg-malig,breast,breast-quad,irradiat
0,30-39,premeno,30-34,0-2,no,3,left,left_low,no
1,40-49,premeno,20-24,0-2,no,2,right,right_up,no
2,40-49,premeno,20-24,0-2,no,2,left,left_low,no
3,60-69,ge40,15-19,0-2,no,2,right,left_up,no
4,40-49,premeno,0-4,0-2,no,2,right,right_low,no
...,...,...,...,...,...,...,...,...,...
281,30-39,premeno,30-34,0-2,no,2,left,left_up,no
282,30-39,premeno,20-24,0-2,no,3,left,left_up,yes
283,60-69,ge40,20-24,0-2,no,1,right,left_up,no
284,40-49,ge40,30-34,5-Mar,no,3,left,left_low,no


In [4]:
y

,Class
0,no-recurrence-events
1,no-recurrence-events
2,no-recurrence-events
3,no-recurrence-events
4,no-recurrence-events
...,...
281,recurrence-events
282,recurrence-events
283,recurrence-events
284,recurrence-events


In [5]:
new_data = X

In [6]:
new_data["Class"] = y["Class"].to_list()

In [7]:
new_data

,age,menopause,tumor-size,inv-nodes,node-caps,deg-malig,breast,breast-quad,irradiat,Class
0,30-39,premeno,30-34,0-2,no,3,left,left_low,no,no-recurrence-events
1,40-49,premeno,20-24,0-2,no,2,right,right_up,no,no-recurrence-events
2,40-49,premeno,20-24,0-2,no,2,left,left_low,no,no-recurrence-events
3,60-69,ge40,15-19,0-2,no,2,right,left_up,no,no-recurrence-events
4,40-49,premeno,0-4,0-2,no,2,right,right_low,no,no-recurrence-events
...,...,...,...,...,...,...,...,...,...,...
281,30-39,premeno,30-34,0-2,no,2,left,left_up,no,recurrence-events
282,30-39,premeno,20-24,0-2,no,3,left,left_up,yes,recurrence-events
283,60-69,ge40,20-24,0-2,no,1,right,left_up,no,recurrence-events
284,40-49,ge40,30-34,5-Mar,no,3,left,left_low,no,recurrence-events


### Process Nan value

In [8]:
row_nan_count = new_data.isna().sum(axis=1)
print(row_nan_count)

0      0
1      0
2      0
3      0
4      0
      ..
281    0
282    0
283    0
284    0
285    0
Length: 286, dtype: int64


In [9]:
row_nan_count.value_counts()

0    277
1      9
dtype: int64

In [10]:
nonnan_data = new_data.dropna()

In [11]:
nonnan_data

,age,menopause,tumor-size,inv-nodes,node-caps,deg-malig,breast,breast-quad,irradiat,Class
0,30-39,premeno,30-34,0-2,no,3,left,left_low,no,no-recurrence-events
1,40-49,premeno,20-24,0-2,no,2,right,right_up,no,no-recurrence-events
2,40-49,premeno,20-24,0-2,no,2,left,left_low,no,no-recurrence-events
3,60-69,ge40,15-19,0-2,no,2,right,left_up,no,no-recurrence-events
4,40-49,premeno,0-4,0-2,no,2,right,right_low,no,no-recurrence-events
...,...,...,...,...,...,...,...,...,...,...
281,30-39,premeno,30-34,0-2,no,2,left,left_up,no,recurrence-events
282,30-39,premeno,20-24,0-2,no,3,left,left_up,yes,recurrence-events
283,60-69,ge40,20-24,0-2,no,1,right,left_up,no,recurrence-events
284,40-49,ge40,30-34,5-Mar,no,3,left,left_low,no,recurrence-events


In [12]:
import pandas as pd

In [13]:
cat_columns = nonnan_data.select_dtypes(['object']).columns 
nonnan_data[cat_columns] = nonnan_data[cat_columns].apply(lambda x: pd.factorize(x)[0])
# convert boolean to numberic values
bool_columns = nonnan_data.select_dtypes(['boolean']).columns
nonnan_data[bool_columns] = nonnan_data[bool_columns].replace({True: 1, False: 0})

/tmp/ipykernel_2769698/2148826127.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nonnan_data[cat_columns] = nonnan_data[cat_columns].apply(lambda x: pd.factorize(x)[0])


In [14]:
nonnan_data

,age,menopause,tumor-size,inv-nodes,node-caps,deg-malig,breast,breast-quad,irradiat,Class
0,0,0,0,0,0,3,0,0,0,0
1,1,0,1,0,0,2,1,1,0,0
2,1,0,1,0,0,2,0,0,0,0
3,2,1,2,0,0,2,1,2,0,0
4,1,0,3,0,0,2,1,3,0,0
...,...,...,...,...,...,...,...,...,...,...
281,0,0,0,0,0,2,0,2,0,1
282,0,0,1,0,0,3,0,2,1,1
283,2,1,1,0,0,1,1,2,0,1
284,1,1,0,3,0,3,0,0,0,1


In [ ]:
nonnan_data.to_csv("/workspace/tripx/MCS/xai_causality/dataset/breast_cancer_uci.csv")

In [3]:
!pip install scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 995.4 kB/s eta 0:00:000:0100:01
  Using cached threadpoolctl-3.2.0-py3-none-any.whl (15 kB)


In [1]:
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score
import pandas as pd
from sklearn.svm import SVC

In [2]:
data_path = "/workspace/tripx/MCS/xai_causality/dataset/breast_cancer_uci.csv"
data = pd.read_csv(data_path)

In [3]:
data = data.to_numpy()

In [4]:
X = data[:,1:-2]
y = data[:,-1]

In [5]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

In [6]:
X_train

array([[0, 0, 8, ..., 3, 0, 0],
       [2, 1, 6, ..., 1, 1, 0],
       [3, 1, 6, ..., 1, 1, 2],
       ...,
       [1, 0, 1, ..., 2, 0, 2],
       [3, 1, 7, ..., 3, 0, 0],
       [3, 0, 4, ..., 2, 1, 3]])

In [7]:
y_train

array([1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0,
       0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0,
       0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1,
       0])

In [ ]:
X_test

In [9]:
clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))
clf.fit(X_train, y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svc', SVC(gamma='auto'))])

In [10]:
y_pred = clf.predict(X_test)

In [11]:
from sklearn.metrics import classification_report
target_names = ["No", "Yes"]
print(classification_report(y_test, y_pred, target_names=target_names))

              precision    recall  f1-score   support

          No       0.69      0.92      0.79        37
         Yes       0.57      0.21      0.31        19

    accuracy                           0.68        56
   macro avg       0.63      0.56      0.55        56
weighted avg       0.65      0.68      0.63        56



In [ ]:
from sklearn import svm, datasets
from sklearn.model_selection import GridSearchCV
iris = datasets.load_iris()
parameters = {'kernel':('linear', 'rbf'), 'C':[1, 10]}
svc = svm.SVC()
clf = GridSearchCV(svc, parameters)
clf.fit(iris.data, iris.target)
sorted(clf.cv_results_.keys())

In [10]:
temp = clf.decision_function(iris.data)

In [12]:
temp

array([[ 2.24627744,  1.2980152 , -0.30616012],
       [ 2.23781119,  1.29663601, -0.30453043],
       [ 2.24548583,  1.2968967 , -0.30542241],
       [ 2.23591041,  1.29559718, -0.30371626],
       [ 2.24795778,  1.29794432, -0.3062846 ],
       [ 2.23752685,  1.29708102, -0.30477998],
       [ 2.2434869 ,  1.2959543 , -0.30466613],
       [ 2.24100113,  1.2972027 , -0.30517051],
       [ 2.23661182,  1.2950946 , -0.30347408],
       [ 2.2375786 ,  1.29702254, -0.30474832],
       [ 2.24558626,  1.29867717, -0.30648862],
       [ 2.23718064,  1.2962742 , -0.30425006],
       [ 2.23997815,  1.29699621, -0.30494992],
       [ 2.2512266 ,  1.29694666, -0.30605703],
       [ 2.25842469,  1.3010311 , -0.30912448],
       [ 2.25245732,  1.29938656, -0.30757506],
       [ 2.25194914,  1.29863279, -0.30709033],
       [ 2.24452273,  1.29742852, -0.3056412 ],
       [ 2.23723948,  1.2982319 , -0.30547458],
       [ 2.24582594,  1.29755577, -0.30584425],
       [ 2.23172213,  1.29740176, -0.304

### OV Dataset

In [1]:
import pandas as pd

In [2]:
data_path = '/dataset/PANCAN/OV_gene_filter.csv'

In [3]:
ov_data = pd.read_csv(data_path)

In [4]:
ov_data

,Unnamed: 0,BRCA1,CDK12,KRAS,NF1,NRAS,RB1,TP53,ZNF133,disease
0,0,287.181420,763.629592,1084.822497,793.821043,1966.832379,1228.185560,4366.762064,431.361256,1
1,1,63.554002,913.661396,1500.292432,1080.977586,1199.540058,935.214062,673.875925,323.202295,1
2,2,10.822760,1253.248243,1153.482062,1438.677128,1719.130917,1015.142839,412.595183,366.707568,1
3,3,353.307926,884.187831,706.406991,838.802260,2329.127521,834.386739,3031.899487,1281.558390,1
4,4,279.946044,838.837426,1429.197374,175.783778,2690.918542,261.067772,3071.559875,290.063072,1
...,...,...,...,...,...,...,...,...,...,...
585,585,431.085000,1238.030000,1217.010000,1795.700000,1572.340000,855.327000,2841.150000,593.842000,0
586,586,210.169000,853.559000,680.000000,1755.930000,946.441000,562.034000,2410.850000,429.830000,0
587,587,1065.900000,1719.520000,903.408000,2249.230000,2195.720000,2062.520000,1633.620000,153.478000,0
588,588,387.585000,8566.970000,1249.180000,1568.760000,1515.300000,501.634000,1967.920000,365.904000,0


In [5]:
ov_data = ov_data.drop(['Unnamed: 0'], axis=1)

In [6]:
ov_data

,BRCA1,CDK12,KRAS,NF1,NRAS,RB1,TP53,ZNF133,disease
0,287.181420,763.629592,1084.822497,793.821043,1966.832379,1228.185560,4366.762064,431.361256,1
1,63.554002,913.661396,1500.292432,1080.977586,1199.540058,935.214062,673.875925,323.202295,1
2,10.822760,1253.248243,1153.482062,1438.677128,1719.130917,1015.142839,412.595183,366.707568,1
3,353.307926,884.187831,706.406991,838.802260,2329.127521,834.386739,3031.899487,1281.558390,1
4,279.946044,838.837426,1429.197374,175.783778,2690.918542,261.067772,3071.559875,290.063072,1
...,...,...,...,...,...,...,...,...,...
585,431.085000,1238.030000,1217.010000,1795.700000,1572.340000,855.327000,2841.150000,593.842000,0
586,210.169000,853.559000,680.000000,1755.930000,946.441000,562.034000,2410.850000,429.830000,0
587,1065.900000,1719.520000,903.408000,2249.230000,2195.720000,2062.520000,1633.620000,153.478000,0
588,387.585000,8566.970000,1249.180000,1568.760000,1515.300000,501.634000,1967.920000,365.904000,0


In [7]:
row_nan_count = ov_data.isna().sum(axis=1)
row_nan_count.value_counts()

0    590
dtype: int64

In [8]:
ov_data

,BRCA1,CDK12,KRAS,NF1,NRAS,RB1,TP53,ZNF133,disease
0,287.181420,763.629592,1084.822497,793.821043,1966.832379,1228.185560,4366.762064,431.361256,1
1,63.554002,913.661396,1500.292432,1080.977586,1199.540058,935.214062,673.875925,323.202295,1
2,10.822760,1253.248243,1153.482062,1438.677128,1719.130917,1015.142839,412.595183,366.707568,1
3,353.307926,884.187831,706.406991,838.802260,2329.127521,834.386739,3031.899487,1281.558390,1
4,279.946044,838.837426,1429.197374,175.783778,2690.918542,261.067772,3071.559875,290.063072,1
...,...,...,...,...,...,...,...,...,...
585,431.085000,1238.030000,1217.010000,1795.700000,1572.340000,855.327000,2841.150000,593.842000,0
586,210.169000,853.559000,680.000000,1755.930000,946.441000,562.034000,2410.850000,429.830000,0
587,1065.900000,1719.520000,903.408000,2249.230000,2195.720000,2062.520000,1633.620000,153.478000,0
588,387.585000,8566.970000,1249.180000,1568.760000,1515.300000,501.634000,1967.920000,365.904000,0


### STAD

In [ ]:
import pandas as pd
data_path = '/dataset/PANCAN/STAD_gene_filter.csv'

In [ ]:
stad_data = pd.read_csv(data_path)

In [ ]:
row_nan_count = stad_data.isna().sum(axis=1)
row_nan_count.value_counts()

In [ ]:
row_nan_count

In [ ]:
stad_data

# Regression Data

### Diabetes

In [1]:
from sklearn import datasets, ensemble


In [2]:
diabetes = datasets.load_diabetes()
X, y = diabetes.data, diabetes.target

In [ ]:
print(diabetes.feature_names)

### Fetch California Housing

In [ ]:
from sklearn.datasets import fetch_california_housing
X, y = fetch_california_housing(return_X_y=True, as_frame=True)
n_samples, n_features = X.shape

In [ ]:
print(f"The dataset consists of {n_samples} samples and {n_features} features")
